In [1]:
from preprocess import dataset,get_Kfold
from sklearn.preprocessing import MinMaxScaler
from models import MLP
from torch import optim
from torch.utils.data import DataLoader,Dataset
import torch 
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import json
import random
from TrainandTest import train_MLP,test_MLP

def same_seeds(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

same_seeds(114514)

In [2]:
def get_models(model):
    if model=='MLP':
        return MLP(5,64)

def train_epoch(train_loader,model,criterion,optimizer,device):
    losses = []
    for i, (inputs, labels) in enumerate(train_loader):
        labels = labels.to(device)
        inputs = inputs.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels.unsqueeze(1))

        loss.backward()
        optimizer.step()
    losses.append(loss.item())
    return sum(losses)/len(losses)

In [16]:
with open('config.json','r',encoding='utf-8')as f:
    config_list = json.load(f)
#device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = 'cpu'
file = './data/jiji small126.xlsx'
batch_size = 300

for config in config_list:
    Type = config['Type']
    N_size = config['N_size']
    num_epoch = config['num_epoch']
    lr = config['lr']
    k = config['k']
    alpha,beta = config['alpha'],config['beta']
    
    K_fold_data = get_Kfold(file,2)
    for k_step,(trian_data,test_data) in enumerate(K_fold_data):
        print("-----------[ K:{k_step+1} Type:{Type} N_size:{N_size} ]-----------")
        model = get_models(config['model']).to(device)
        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(),lr=lr)
        lr_scheduler = optim.lr_scheduler.LambdaLR(optimizer,lambda x: 1 - x/num_epoch,last_epoch=-1)
        
        scaler = MinMaxScaler()
        trian_data = dataset(trian_data,Type,N_size)
        test_data = dataset(test_data,Type,N_size)
 #       trian_data = scaler.fit_transform(trian_data)
 #       test_data = scaler.transform(test_data)
        train_loader = DataLoader(dataset=trian_data,batch_size=batch_size)
        test_loader = DataLoader(dataset=test_data,batch_size=batch_size)
        train_MLP(train_loader=train_loader,model=model,criterion=criterion,optimizer=optimizer,epoch = 1000,fold = k_step,lr_scheduler=lr_scheduler)
        model.load_state_dict(torch.load(f'./model_fold_{k_step + 1}.ckpt'))
        model.eval()
        test_losses = []
        with torch.no_grad():  
            for (inputs, labels) in enumerate(test_loader):
                labels = labels 
                outputs = model(inputs)
                print(labels,outputs)
                loss = criterion(outputs, labels.unsqueeze(1))
            test_losses.append(loss.item())
            test_loss = sum(test_losses)/len(test_losses)
            rmse = test_loss**0.5
            print(f'Test Loss: {test_loss:.10f}, Test RMSE: {rmse:.10f}') 
    

-----------[ K:{k_step+1} Type:{Type} N_size:{N_size} ]-----------
loss:6.9553070068359375 saving...
loss:1.6970878839492798 saving...
loss:1.6833913326263428 saving...
loss:1.5328588485717773 saving...
loss:0.9129649996757507 saving...
loss:0.7964922189712524 saving...
loss:0.612503707408905 saving...
loss:0.5151007771492004 saving...
loss:0.33268097043037415 saving...
loss:0.11701059341430664 saving...
loss:0.06782921403646469 saving...
loss:0.020726999267935753 saving...
loss:0.017559058964252472 saving...
loss:0.016195999458432198 saving...
loss:0.015904607251286507 saving...
loss:0.00934223085641861 saving...
loss:0.004964521620422602 saving...
loss:0.00226382864639163 saving...
loss:0.0009549474925734103 saving...
loss:0.0008553307852707803 saving...
loss:0.0008150510839186609 saving...
loss:0.0007965481490828097 saving...
loss:0.00040174214518629014 saving...
loss:0.0002625538036227226 saving...
loss:0.0002505354641471058 saving...
loss:0.00024850599584169686 saving...
loss:0.00

loss:6.086298753871233e-07 saving...
loss:5.819295552100812e-07 saving...
loss:5.591409717453644e-07 saving...
loss:5.391007675825676e-07 saving...
loss:5.163385026207834e-07 saving...
loss:4.946339799971611e-07 saving...
loss:4.7511585421489144e-07 saving...
loss:4.557957993256423e-07 saving...
loss:4.3888167056138627e-07 saving...
loss:4.2194923821625707e-07 saving...
loss:4.029421347695461e-07 saving...
loss:3.8922283351894293e-07 saving...
loss:3.751608517177374e-07 saving...
loss:3.5583596513788507e-07 saving...
loss:3.447986216542631e-07 saving...
loss:3.3239447816413303e-07 saving...
loss:3.13764957127205e-07 saving...
loss:3.044533229967783e-07 saving...
loss:2.944957202544174e-07 saving...
loss:2.7670969870996487e-07 saving...
loss:2.690838130092743e-07 saving...
loss:2.6496417149246554e-07 saving...
loss:2.479471845617809e-07 saving...
loss:2.3311250174629095e-07 saving...
loss:2.326241173022936e-07 saving...
loss:2.274564394610934e-07 saving...
loss:2.063671473706563e-07 sav

TypeError: linear(): argument 'input' (position 1) must be Tensor, not int

In [15]:
print(test_losses)

[0.0004652027855627239]
